##### *Reminder - use daily log to track progress

In [ ]:
!pip install librosa

In [22]:
import pandas as pd
import os, pathlib
from sklearn.model_selection import train_test_split
import librosa

In [ ]:
!sudo apt-get install wget unzip

In [ ]:
# Download training data
!mkdir -p data && cd data && wget https://irmas-dataset.s3-eu-west-1.amazonaws.com/IRMAS-TrainingData.zip && unzip IRMAS-TrainingData.zip 

In [ ]:
#Download testing data (Around 2GB each)
wget https://irmas-dataset.s3-eu-west-1.amazonaws.com/IRMAS-TestingData-Part1.zip && IRMAS-TestingData-Part1.zip
wget https://irmas-dataset.s3-eu-west-1.amazonaws.com/IRMAS-TestingData-Part2.zip && IRMAS-TestingData-Part2.zip
wget https://irmas-dataset.s3-eu-west-1.amazonaws.com/IRMAS-TestingData-Part3.zip && IRMAS-TestingData-Part3.zip    

IRMAS dataset (training)
 
Audio files: 6705 audio files in 16 bit stereo wav format sampled at 44.1kHz. They are excerpts of 3 seconds from more than 2000 distinct recordings. 

Annotations: The annotation of the predominant instrument of each excerpt is both in the name of the containing folder, and in the file name: cello (cel), clarinet (cla), flute (flu), acoustic guitar (gac), electric guitar (gel), organ (org), piano (pia), saxophone (sax), trumpet (tru), violin (vio), and human singing voice (voi). The number of files per instrument are: cel(388), cla(505), flu(451), gac(637), gel(760), org(682), pia(721), sax(626), tru(577), vio(580), voi(778). 

Additionally, some of the files have annotations in the filename regarding the presence ([dru]) or non presence([nod]) of drums, and the musical genre: country-folk ([cou_fol]), classical ([cla]), pop-rock ([pop-roc]), latin-soul ([lat-sou]).

In [2]:
IRMAS_TRAINING = 'data/IRMAS-TrainingData'
base_path = pathlib.Path(IRMAS_TRAINING)
classes, paths = [], []

for p in base_path.glob('*/*'):
    relative_path = p.relative_to(base_path)
    classes.append(str(relative_path.parent))
    paths.append(p)
    
#classes = set(classes)
df = pd.DataFrame({"tags": classes, "wav_path": paths}).sample(frac=1).reset_index()
#df_training.piano = df.tags.map({'pia': True, '': 0})
#df.tags = df.tags.map({'pia': True, })

df.head(5)

,index,tags,wav_path
0,2592,sax,Data/IRMAS-TrainingData/sax/081__[sax][nod][cl...
1,6001,gel,Data/IRMAS-TrainingData/gel/[gel][pop_roc]0888...
2,2764,org,Data/IRMAS-TrainingData/org/[org][jaz_blu]1232...
3,3792,cla,Data/IRMAS-TrainingData/cla/[cla][jaz_blu]0325...
4,4673,vio,Data/IRMAS-TrainingData/vio/[vio][jaz_blu]2108...


In [3]:
df.shape

(6705, 3)

In [4]:
set(classes)

{'cel', 'cla', 'flu', 'gac', 'gel', 'org', 'pia', 'sax', 'tru', 'vio', 'voi'}

In [23]:
df_train, df_test = train_test_split(df, test_size=0.3)

(4693, 3)

(2012, 3)

In [27]:
# Test if files are loading
successful, corrupted = [], []
len_x, sample_rates = [], []
for p in df.wav_path:
    try:
        x, sr = librosa.load(p, sr=None)
        successful.append(p)
        len_x.append(len(x))
        sample_rates.append(sr)
    except:
        corrupted.append(p)
#       print(p)
###df = df[~df.wav_path.isin(corrupted)]
len(successful), len(corrupted)

(6705, 0)

In [29]:
set(len_x)

{132299}

In [30]:
set(sample_rates)

{44100}

In [31]:
132299/44100

2.9999773242630385

In [20]:
sr

44100

In [ ]:
df_test.shape

In [ ]:
class InstrumentClassificationDataset(Dataset):
    def __init__(self, annotations):
        super().__init__()
        self.annotations = annotations.copy()
#        self.image_transform = image_transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        sound = self.load_from_disk(index)
        label = self.load_label(index)
#        Xi = self.image_transform(Xi)
        return sound, label

    def load_to_librosa(self, uri):
        image = PIL.Image.open(uri)
        return image

    def load_from_disk(self, index):
        image_path = df.iloc[index].wav_path 
        return self.load_to_librosa(wav_path)

    def load_label(self, index):
        label = df.iloc[index].tags
        return label

In [ ]:
class BaseSampler(Sampler):
    def __init__(self, df, list_of_instruments, n_samples):
        self.df = df.copy()
        self.n_samples = n_samples
        self.instruments = list_of_instruments
        #df = df[self.instruments]
        df = df[df['tags'].isin(inst)]
        
    def __iter__(self):
        return iter(self._get_sample())
        
    def __len__(self):
        return self.n_samples
    
    def _get_samples(self, ):
        return np.random.choice(len(self.df), self.n_samples, replace=False)

In [32]:
df

,index,tags,wav_path
0,2592,sax,Data/IRMAS-TrainingData/sax/081__[sax][nod][cl...
1,6001,gel,Data/IRMAS-TrainingData/gel/[gel][pop_roc]0888...
2,2764,org,Data/IRMAS-TrainingData/org/[org][jaz_blu]1232...
3,3792,cla,Data/IRMAS-TrainingData/cla/[cla][jaz_blu]0325...
4,4673,vio,Data/IRMAS-TrainingData/vio/[vio][jaz_blu]2108...
...,...,...,...
6700,5913,gel,Data/IRMAS-TrainingData/gel/064__[gel][dru][ja...
6701,1287,voi,Data/IRMAS-TrainingData/voi/261__[voi][dru][co...
6702,501,pia,Data/IRMAS-TrainingData/pia/[pia][jaz_blu]1503...
6703,5266,flu,Data/IRMAS-TrainingData/flu/093__[flu][dru][po...


In [34]:
inst = ['sax', 'pia']

In [38]:
df_inst = df[df['tags'].isin(inst)]

In [41]:
df_inst

,index,tags,wav_path
0,2592,sax,Data/IRMAS-TrainingData/sax/081__[sax][nod][cl...
5,359,pia,Data/IRMAS-TrainingData/pia/152__[pia][dru][po...
12,2698,sax,Data/IRMAS-TrainingData/sax/094__[sax][dru][ja...
13,698,pia,Data/IRMAS-TrainingData/pia/[pia][cla]1447__2.wav
16,2426,sax,Data/IRMAS-TrainingData/sax/[sax][jaz_blu]1762...
...,...,...,...
6647,177,pia,Data/IRMAS-TrainingData/pia/[pia][cla]1352__1.wav
6654,185,pia,Data/IRMAS-TrainingData/pia/[pia][pop_roc]1303...
6687,2669,sax,Data/IRMAS-TrainingData/sax/[sax][pop_roc]1604...
6699,262,pia,Data/IRMAS-TrainingData/pia/[pia][jaz_blu]1508...
